In [1]:
import os
import torch
device='cuda' if torch.cuda.is_available() else 'cpu' 
from tqdm.notebook import tqdm
from transformers import BertForSequenceClassification, BertTokenizer,BertConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,load_metric
import torch.nn.functional as F
os.environ["WANDB_DISABLED"] = "true"
import torch.nn as nn
import copy

In [2]:
train_dataset = load_dataset('glue', 'sst2', split='train') # challenge, sst2
val_dataset = load_dataset('glue', 'sst2', split='validation')
test_dataset = load_dataset('glue', 'sst2', split='test')

Reusing dataset glue (/home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [3]:
train_dataset = train_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
val_dataset = val_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
test_dataset = test_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

val_dataset = val_dataset.remove_columns(['label'])
test_dataset = test_dataset.remove_columns(['label'])
train_dataset = train_dataset.remove_columns(['label'])

Loading cached processed dataset at /home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7cf2378cc10a6d21.arrow
Loading cached processed dataset at /home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-80a8390bc04583f0.arrow
Loading cached processed dataset at /home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-63ad18a8c03484fc.arrow


In [4]:
# tokenizer = BertTokenizer.from_pretrained('saved_model')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
MAX_LENGTH = 128
train_dataset = train_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
val_dataset = val_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)
test_dataset = test_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)

Loading cached processed dataset at /home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-557e3dbb80f86843.arrow
Loading cached processed dataset at /home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3d2ecae1eafe9952.arrow
Loading cached processed dataset at /home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-61bb57b2f450d330.arrow


In [6]:
train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [7]:
train_dataset

Dataset({
    features: ['sentence', 'idx', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 67349
})

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [9]:
def deleteEncodingLayers(model, num_layers_to_keep):  # must pass in the full bert model
    oldModuleList = model.bert.encoder.layer
    newModuleList = nn.ModuleList()

    # Now iterate over all layers, only keepign only the relevant layers.
    for i in range(0, len(num_layers_to_keep)):
        newModuleList.append(oldModuleList[i])

    # create a copy of the model, modify it with the new list, and return
    copyOfModel = copy.deepcopy(model)
    copyOfModel.bert.encoder.layer = newModuleList

    return copyOfModel

In [24]:
teacher_model = BertForSequenceClassification.from_pretrained('./bert_main_sst_model', return_dict=True)
student =  deleteEncodingLayers(teacher_model, [1,5,8,12])

loading configuration file ./bert_main_sst_model/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./bert_main_sst_model/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceCla

In [25]:
student

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [26]:
training_args = TrainingArguments(
    output_dir='./results',      
    overwrite_output_dir = 'True',#output directory
    learning_rate=1e-5,
    num_train_epochs=50,
    warmup_steps = 300,
    weight_decay=0.01,
    per_device_train_batch_size=64,                #batch size per device during training
    per_device_eval_batch_size=64,                #batch size for evaluation
    logging_dir='./logs',            
    logging_steps=100,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    load_best_model_at_end=False,
    save_total_limit = 1,
    save_strategy = "no",
    seed=42,
    evaluation_strategy="epoch"
)
class CustomTrainer(Trainer):
    def loss_fn_kd(self, outputs, labels, teacher_outputs):
        """
        Compute the knowledge-distillation (KD) loss given outputs, labels.
        "Hyperparameters": temperature and alpha
        NOTE: the KL Divergence for PyTorch comparing the softmaxs of teacher
        and student expects the input tensor to be log probabilities! See Issue #2
        """
        params = {
                "alpha": 0.95,
                "temperature": 6,
        }
        alpha = params['alpha']
        T = params['temperature']
        KD_loss = nn.KLDivLoss()(F.log_softmax(outputs/T, dim=1),
                                 F.softmax(teacher_outputs/T, dim=1)) * (alpha * T * T) + \
                  F.cross_entropy(outputs, labels) * (1. - alpha)

        return KD_loss
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        teacher_model.eval().cuda()
        teacher_outputs = teacher_model(**inputs)
        student_outputs = model(**inputs)
        
        teacher_logits = teacher_outputs.get("logits")
        student_logits = student_outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss = self.loss_fn_kd(student_logits.view(-1, self.model.config.num_labels), labels.view(-1), teacher_logits.view(-1, self.model.config.num_labels))
        return (loss, student_outputs) if return_outputs else loss
    
trainer = CustomTrainer(
    model=student,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [27]:
train_out = trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 67349
  Num Epochs = 50
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 52650
/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/torch/nn/functional.py:2886: UserWarning: reduction: 'mean' divides the tota

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.024600,0.023625,0.803899,0.803682,0.804163,0.803559
2,0.017000,0.019480,0.833716,0.833531,0.834049,0.833386
3,0.014400,0.018173,0.855505,0.855376,0.855732,0.855245
4,0.012500,0.017283,0.866972,0.866663,0.868498,0.866380
5,0.010900,0.016309,0.872706,0.872646,0.872712,0.872600
6,0.010000,0.015658,0.878440,0.878440,0.878638,0.878694
7,0.009100,0.016346,0.876147,0.876141,0.876830,0.876568
8,0.008000,0.015940,0.884174,0.884028,0.884761,0.883820
9,0.007300,0.017177,0.877294,0.877072,0.878357,0.876810
10,0.007100,0.017261,0.877294,0.876936,0.879509,0.876600


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 64
/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/torch/nn/functional.py:2886: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message

KeyboardInterrupt: 

In [28]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 64


{'eval_loss': 0.017314976081252098,
 'eval_accuracy': 0.8807339449541285,
 'eval_f1': 0.8807113543091656,
 'eval_precision': 0.8806818181818181,
 'eval_recall': 0.8807779742359182}

In [30]:
student.save_pretrained('./bert_distilled_sst_model/')
tokenizer.save_pretrained('./bert_distilled_sst_model/')

Configuration saved in ./bert_distilled_sst_model/config.json
Model weights saved in ./bert_distilled_sst_model/pytorch_model.bin
tokenizer config file saved in ./bert_distilled_sst_model/tokenizer_config.json
Special tokens file saved in ./bert_distilled_sst_model/special_tokens_map.json


('./bert_distilled_sst_model/tokenizer_config.json',
 './bert_distilled_sst_model/special_tokens_map.json',
 './bert_distilled_sst_model/vocab.txt',
 './bert_distilled_sst_model/added_tokens.json')

In [5]:
############# QUANTIZATION

In [6]:
import torch
import torch.nn as nn
import os
from transformers import BertForSequenceClassification, BertTokenizer,BertConfig

/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (MB):', size/1e6)
    os.remove('temp.p')
    return size

In [8]:
model = BertForSequenceClassification.from_pretrained('bert_main_sst_model', return_dict =True)
quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Linear}, dtype=torch.qint8
)
model_size = print_size_of_model(model,"fp32")
quantized_model_size = print_size_of_model(quantized_model,"int8")

print("{0:.2f} times smaller".format(model_size/quantized_model_size))

model:  fp32  	 Size (MB): 438.016429
model:  int8  	 Size (MB): 181.496197
2.41 times smaller


In [10]:
from datasets import load_dataset,load_metric
MAX_LENGTH = 128
tokenizer = BertTokenizer.from_pretrained('./bert_main_sst_model')

val_dataset = load_dataset('glue', 'sst2', split='validation')
val_dataset = val_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)
val_dataset = val_dataset.remove_columns(['label'])
val_dataset = val_dataset.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=MAX_LENGTH), batched=True)

Reusing dataset glue (/home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-80a8390bc04583f0.arrow
Loading cached processed dataset at /home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-1271d2b78f498c8f.arrow


In [32]:
input_ids =  torch.tensor(val_dataset['input_ids'][:0])
token_type_ids = torch.tensor(val_dataset['token_type_ids'][:0])
attention_mask = torch.tensor(val_dataset['attention_mask'][:0])
dummy_input = [input_ids, attention_mask, token_type_ids]
traced_model = torch.jit.trace(quantized_model, dummy_input, strict=False)

torch.jit.save(traced_model, "bert_quantized8_sst_model/model.pt")

/state/partition1/job-19337151/ipykernel_1806277/3651675027.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids =  torch.tensor(val_dataset['input_ids'][:0])
/state/partition1/job-19337151/ipykernel_1806277/3651675027.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token_type_ids = torch.tensor(val_dataset['token_type_ids'][:0])
/state/partition1/job-19337151/ipykernel_1806277/3651675027.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(val_dataset['attention_mask'][:0])
